In [49]:
import pandas as pd
from datetime import date

In [43]:
members = pd.read_csv('data/NSS_GreatCare_Members.csv')
adm = pd.read_csv('data/NSS_Hospital_inpatient_admissions.csv')
pcp = pd.read_csv('data/NSS_PrimaryCare_Office_Visits.csv')

C:\Users\dhoza\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [51]:
members

,Patient ID,Primary Plan ID,Primary Payor ID,Is Patient Alive,Patient Age Integer,Patient Date of Birth,Patient ZIP Code 5-Digit,Count Primary Care Visits in Last 36 Months,Count Visits with Attributee Individual Primary Care Provider i,Count Visits with Attributee Primary Care Provider Group in Las,...,Is CCW Prostate Cancer,Is CCW Rheumatoid Arthritis or Osteoarthritis,Is CCW Stroke or Transient Ischemic Attack,Is CCW ASCVD,Is CCW Anxiety Disorders,Is CCW Depressive Disorders,Is CCW Schizophrenia or Other Psychotic Disorders,Is CCW Bipolar Disorder,Period Start Date,Period End Date
0,DS1|1,DS1|5047W,DS1|1,1,54,1958-11-08,92111.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2013-01-01 00:00:00+00,2013-04-11 00:00:00+00
1,DS1|10,DS1|5047W,DS1|1,1,54,1958-12-10,90024.0,6,1.0,5.0,...,0,0,0,0,1,0,0,0,2012-12-10 00:00:00+00,2013-01-01 00:00:00+00
2,DS1|100,DS1|5047W,DS1|1,1,2,2012-08-04,90008.0,1,1.0,1.0,...,0,0,0,0,0,0,0,0,2015-04-18 00:00:00+00,2015-05-18 00:00:00+00
3,DS1|1000,DS1|5047W,DS1|1,1,6,2003-12-25,94131.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2010-01-01 00:00:00+00,2010-04-09 00:00:00+00
4,DS1|10000,DS1|5047W,DS1|1,1,7,2005-07-15,90301.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2012-08-01 00:00:00+00,2012-10-02 00:00:00+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61514,DS1|9995,DS1|5047W,DS1|1,1,47,1963-07-26,92104.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2011-01-01 00:00:00+00,2011-01-15 00:00:00+00
61515,DS1|9996,DS1|5047X,DS1|1,1,26,1983-04-17,94122.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2010-03-28 00:00:00+00,2010-04-17 00:00:00+00
61516,DS1|9997,DS1|5047W,DS1|1,1,4,2008-12-11,92840.0,1,1.0,1.0,...,0,0,0,0,0,0,0,0,2013-01-01 00:00:00+00,2013-02-01 00:00:00+00
61517,DS1|9998,DS1|5047X,DS1|1,1,63,1954-05-23,94939.0,1,1.0,1.0,...,0,0,0,0,0,0,0,0,2017-11-05 00:00:00+00,2018-05-23 00:00:00+00


In [46]:
#choosing columns needed
dayreadmin = adm[['Patient ID','Facility Provider ID','Encounter Start Date','Encounter End Date']]

In [47]:
##renaming columns to be one "word"
dayreadmin.columns = ['patient_id','facility_id','start_date','end_date']
dayreadmin

,patient_id,facility_id,start_date,end_date
0,DS1|10033,DS1|NPI|1306890389,2013-01-12,2013-01-14
1,DS1|10043,DS1|NPI|1902803315,2014-01-05,2014-01-12
2,DS1|10073,DS1|NPI|1902803315,2012-06-13,2012-06-21
3,DS1|10101,DS1|NPI|1427055839,2012-12-06,2012-12-09
4,DS1|10118,DS1|NPI|1992818256,2015-03-11,2015-03-14
...,...,...,...,...
6208,DS1|9979,DS1|NPI|1689772592,2015-02-06,2015-02-11
6209,DS1|9981,DS1|NPI|1427055839,2014-08-07,2014-08-10
6210,DS1|9994,DS1|NPI|1689608150,2015-11-21,2015-11-23
6211,DS1|9994,DS1|NPI|1689608150,2015-11-24,2015-11-27


In [48]:
##PULLING ALL DUPLICATED PT ID
mask = dayreadmin.patient_id.duplicated(keep=False)
print (dayreadmin[mask])

     patient_id         facility_id  start_date    end_date
7     DS1|10132  DS1|NPI|1366545311  2013-08-14  2013-08-16
8     DS1|10132  DS1|NPI|1386741635  2013-03-30  2013-04-01
17    DS1|10253  DS1|NPI|1215987532  2011-10-15  2011-11-13
18    DS1|10253  DS1|NPI|1396725677  2010-06-02  2010-06-06
19    DS1|10253  DS1|NPI|1689772592  2010-04-02  2010-04-16
...         ...                 ...         ...         ...
6201   DS1|9943  DS1|NPI|1184722779  2013-10-01  2013-10-16
6202   DS1|9943  DS1|NPI|1184722779  2013-10-25  2013-10-28
6203   DS1|9943  DS1|NPI|1184722779  2014-10-28  2014-10-30
6210   DS1|9994  DS1|NPI|1689608150  2015-11-21  2015-11-23
6211   DS1|9994  DS1|NPI|1689608150  2015-11-24  2015-11-27

[2382 rows x 4 columns]


In [ ]:
day30readm=mask['end_date'] - mask[]